# Application On all sources

## Imports

In [6]:
%load_ext autoreload
%autoreload 2
import numpy as np 
from matplotlib import pyplot as plt 
import pandas as pd 
import seaborn as sns
import joblib


In [12]:
from utilities import make_model
from choices import param_dict 
from utilities import deets

## Load

### Load Model

In [11]:
model = joblib.load('models/lightGBM-final.pkl')
#model
clf = model.clf
clf 

LGBMClassifier(class_weight={'AGN': 1.2524978967563674,
                             'CV': 25.745323836808723,
                             'HMXB': 2.0562192560551287,
                             'LMXB': 43.41041115489145,
                             'PULSAR': 208.24450024014098,
                             'STAR': 1.2132045326966958,
                             'ULX': 12.87770640060345,
                             'YSO': 1.5988527422270495},
               force_col_wise=True, is_unbalance=True, metric=['auc_mu'],
               num_class=8, objective='multiclass', random_state=42,
               sparse=True, verbosity=0)

### Load Data

In [15]:
x = pd.read_csv('compiled_data_v3/x_phot_minmax.csv' , index_col='name')
x_col = x.columns.to_list()
i = pd.read_csv('compiled_data_v3/id_frame.csv' , index_col='name')
df = pd.merge(i,x , left_index =True , right_index =True )
df = df[df['offset']<1.01]
feat_to_drop = np.unique(param_dict['hardness'] + param_dict['IRAC']+[
    '0p5_2csc' , '2-10 keV (XMM)' , '1_2_csc' , '0p5_8_csc'
]+['var_inter_hard_prob_hs', 'ks_intra_prob_b', 'var_inter_hard_sigma_hm', 'var_inter_hard_prob_ms', 'var_inter_hard_prob_hm',] )
classes = ['AGN' ,'STAR' ,'YSO' ,'HMXB' , 'LMXB' , 'ULX' ,'CV' , 'PULSAR']
df = df[df['class'].isin(classes)]
#from sklearn.preprocessing import LabelEncoder
#y_int = LabelEncoder().fit_transform(df['class'])
#deets(df,1)
u = pd.read_csv('compiled_data_v3/x_phot_minmax-unid-var-src.csv' , index_col='name')
u = u.drop(columns=feat_to_drop)
u.columns = u.columns.str.replace(' ' ,'_' , regex=False)
x = df[x_col]
x = x.drop(columns=feat_to_drop)
x.columns = x.columns.str.replace(' ' ,'_' , regex=False)
y = df['class']
deets(x)
deets(u)

_____________________________________________________
------------------------------
Number of Objects : 7703
Number of Columns : 41
_____________________________________________________
_____________________________________________________
------------------------------
Number of Objects : 39893
Number of Columns : 41
_____________________________________________________


In [16]:
# u = []
# ui = []
# for i in range(8):
#     u.append(pd.read_csv(f'compiled_data_v3/unid-non-var-chunks/x-chunk-{i}.csv' , index_col='name'))
#     ui.append(pd.read_csv(f'compiled_data_v3/unid-non-var-chunks/id-chunk-{i}.csv' , index_col='name'))
# u = pd.concat(u)
# ui = pd.concat(ui)
ui = pd.read_csv('compiled_data_v3/id_frame-unid-var-src.csv' , index_col='name')
u = pd.read_csv('compiled_data_v3/x_phot_minmax-unid-var-src.csv' , index_col='name')
u = u.drop(columns=feat_to_drop)
u.columns = u.columns.str.replace(' ' ,'_' , regex=False)
u
u = u.reset_index()
# remove sources which were in training set
u = u[~u['name'].isin(x.index.to_list())].set_index('name')
u

,gal_b2,gal_l2,var_inter_prob_b,var_inter_sigma_b,var_intra_prob_b,var_inter_index_b,kp_intra_prob_b,var_intra_index_b,h-csc,b-csc,...,Rp,B-R,G-J,G-W2,Bp-H,Bp-W3,Rp-K,J-H,J-W1,W1-W2
name,,,,,,,,,,,,,,,,,,,,,
2CXO J004829.5-732959,2.560959,8.422809,NaN,NaN,2.363359,NaN,9.949403,0.0,3.304254e-09,0.002147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J005449.0-725128,2.525176,8.405124,NaN,NaN,0.900227,NaN,1.221561,0.0,4.201018e-07,0.011944,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J010049.1-731526,2.548291,8.388764,NaN,NaN,2.694112,NaN,4.333967,0.0,8.611031e-08,0.003832,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J010049.8-731045,2.543942,8.388578,NaN,NaN,3.098912,NaN,3.680766,0.0,1.742587e-06,0.009243,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J010647.7-730940,2.544536,8.371916,NaN,NaN,9.693387,NaN,9.994160,6.0,5.241312e-08,0.003135,...,7.329417,3.418280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2CXO J220433.1-495916,2.153631,9.602391,6.284069,0.008090,0.848783,5.00,9.398184,0.0,2.267642e-07,0.035677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J220508.3-495935,2.148799,9.600738,7.232708,0.007515,7.897757,6.25,9.373067,2.0,8.028243e-08,0.041673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J220544.0-495505,2.142337,9.602388,7.905990,0.007822,NaN,6.25,NaN,NaN,1.863801e-07,0.016260,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Make Predictions

In [18]:
from utilities import softmax , norm_prob
pred_prob = (clf.predict_proba(u))
pred_prob_df = pd.DataFrame(pred_prob , columns=[f'prob_{el}' for el in clf.classes_] , index = u.index.to_list())
pred_prob_df

,prob_AGN,prob_CV,prob_HMXB,prob_LMXB,prob_PULSAR,prob_STAR,prob_ULX,prob_YSO
2CXO J004829.5-732959,0.006866,0.003271,0.854534,0.000017,0.002261,0.115568,0.017461,0.000021
2CXO J005449.0-725128,0.008875,0.016311,0.929717,0.000043,0.000785,0.043891,0.000347,0.000032
2CXO J010049.1-731526,0.008277,0.004818,0.807098,0.000054,0.005160,0.167167,0.007402,0.000025
2CXO J010049.8-731045,0.040766,0.016721,0.526201,0.000443,0.003861,0.407359,0.004558,0.000092
2CXO J010647.7-730940,0.004241,0.009808,0.683742,0.002000,0.008044,0.291919,0.000168,0.000077
...,...,...,...,...,...,...,...,...
2CXO J220433.1-495916,0.032561,0.233294,0.671695,0.000025,0.000336,0.007829,0.054160,0.000100
2CXO J220508.3-495935,0.017150,0.063122,0.913425,0.000210,0.000860,0.001775,0.003403,0.000054
2CXO J220544.0-495505,0.089127,0.091377,0.801407,0.000579,0.000613,0.013682,0.003008,0.000208
2CXO J220555.6-500837,0.000871,0.008760,0.926816,0.000044,0.000296,0.000558,0.062627,0.000027
